In [1]:
!nvidia-smi

Wed Nov 22 15:11:54 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.06              Driver Version: 545.23.06    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A40                     Off | 00000000:AF:00.0 Off |                    0 |
|  0%   32C    P0              70W / 300W |      4MiB / 46068MiB |      3%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
# import sys,os,os.path
# os.environ['HTTP_PROXY']="http://127.0.0.1:8098"
# os.environ['HTTPS_PROXY']="http://127.0.0.1:8098"

In [3]:
import pandas as pd

In [4]:
excel_1 = pd.read_excel("./data/(warebot1.0)工廠端匯總Q&A.xlsx", sheet_name=None)

In [5]:
excel_2 = pd.read_excel("./data/(warebot1.0)平台開發Q&A.xlsx", sheet_name=None)

In [6]:
excel_3 = pd.read_excel("./data/(warebot1.0)交管Q&A.xlsx", sheet_name=None)

In [7]:
all_excels = [
    excel_1,
    excel_2,
    excel_3
]
map_sheets = map(lambda excel_i: pd.concat(excel_i.values(), axis=0, ignore_index=True), all_excels)


In [8]:
from functools import reduce
data = reduce(lambda x, y: pd.concat([x,y], axis=0, ignore_index=True), list(map_sheets))

In [9]:
data = data[["问题","答案"]]

In [10]:
data.shape

(374, 2)

In [11]:
from datasets import Dataset
dataset = Dataset.from_pandas(data)

In [12]:
dataset[0]

{'问题': '无法登入Wareconn员工账号？如何查寻',
 '答案': '進入管理員權限的服務中心點擊基本資料->帳號管理\n1.點擊添加帳號\n2.填寫工號，郵箱等信息，設置密碼，狀態選擇啟用\n3.保存并退出\n4.在帳號管理頁面的搜索欄輸入工號即可查詢'}

In [46]:
!ls -lah /data/python/laby/models/LLM/

total 16G
drwxr-xr-x 4 root root  174 Nov 21 11:28 .
drwxr-xr-x 7 root root  123 Nov 20 08:18 ..
drwxr-xr-x 2 root root  125 Nov 21 11:31 alpaca_cn_trained-model_warebot1.0
-rwxr-xr-x 1 root root  11G Nov 20 08:21 chinese-alpaca-2-13b-16k-q6_k.gguf
drwxrwxrwx 2 laby laby  301 Nov 20 08:18 chinese-alpaca-2-7b
-rw-r--r-- 1 root root 5.3G Nov 20 16:32 chinese-alpaca-2-7b-q6_k.gguf


In [16]:
!ls -lah /root/.cache/huggingface/hub/

total 241M
drwxr-xr-x 15 root root 4.0K Nov 20 12:41 .
drwxr-xr-x  5 root root   61 Oct 24 15:26 ..
drwxr-xr-x  4 root root   94 Nov 13 15:53 .locks
drwxr-xr-x  6 root root   65 Aug 21 09:16 models--bigscience--bloom-1b7
drwxr-xr-x  6 root root   65 Oct 24 18:26 models--bigscience--bloom-7b1
drwxr-xr-x  6 root root   65 Nov 17 13:34 models--FlagAlpha--Llama2-Chinese-7b-Chat
drwxr-xr-x  6 root root   65 Nov 17 12:14 models--FlagAlpha--Llama2-Chinese-7b-Chat_original_copy
drwxr-xr-x  6 root root   65 Nov 20 12:41 models--hfl--chinese-alpaca-2-7b
drwxr-xr-x  6 root root   65 Oct 24 11:46 models--meta-llama--Llama-2-7b-chat-hf
drwxr-xr-x  6 root root   65 Sep  5 00:14 models--shibing624--text2vec-base-chinese
drwxr-xr-x  2 root root    6 Aug 29 13:57 models--TheBloke--Llama-2-7B-GGML
drwxr-xr-x  6 root root   65 Oct 26 13:21 models--tiiuae--falcon-7b
drwxr-xr-x  5 root root   48 Nov 13 09:43 models--vivo-ai--BlueLM-7B-Chat-32K
drwxr-xr-x  5 root root   48 Nov 13 15:54 models--vivo-ai--Blue

In [20]:
# import shutil
# import os
# shutil.copytree(
#     os.path.join(llm_dir, "chinese-alpaca-2-7b"),
#     os.path.join(llm_dir, "chinese-alpaca-2-7b_original_copy"),
# )

In [13]:
import json
import os
from pprint import pprint
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import bitsandbytes as bnb
import pandas as pd
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset
from huggingface_hub import notebook_login
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training,
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    LlamaForCausalLM,
    LlamaTokenizer,
    BitsAndBytesConfig,
)



2023-11-22 15:12:06.222280: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-22 15:12:07.107384: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [14]:
torch.cuda.device_count()

1

In [15]:
torch.device("cuda" if torch.cuda.is_available() else "cpu")

device(type='cuda')

In [16]:
# CPU times: user 54.8 s, sys: 57.3 s, total: 1min 52s
# Wall time: 1h 25min 34s

In [17]:
%%time
# MODEL_NAME = "/data/python/laby/models/LLM/chinese-alpaca-2-7b"
MODEL_NAME = "hfl/chinese-alpaca-2-7b"


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = LlamaForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    # trust_remote_code=True,
    local_files_only=True,
    # quantization_config=bnb_config,
    # temperature=0.0
)

tokenizer = LlamaTokenizer.from_pretrained(MODEL_NAME)
# tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

CPU times: user 56.7 s, sys: 30.7 s, total: 1min 27s
Wall time: 1min


In [18]:
import re
pattern = r'\((\w+)\): Linear'
linear_layers = re.findall(pattern, str(model.modules))
target_modules = list(set(linear_layers))
print(target_modules)

['down_proj', 'o_proj', 'q_proj', 'gate_proj', 'v_proj', 'up_proj', 'k_proj', 'lm_head']


In [19]:
!nvidia-smi

Wed Nov 22 15:13:08 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.06              Driver Version: 545.23.06    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A40                     Off | 00000000:AF:00.0 Off |                    0 |
|  0%   31C    P0              69W / 300W |  26946MiB / 46068MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [20]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )


In [21]:
print_trainable_parameters(model)

trainable params: 6929256448 || all params: 6929256448 || trainable%: 100.0


In [22]:
# model.gradient_checkpointing_enable()
# model = prepare_model_for_kbit_training(model)

In [23]:
# https://github.com/ymcui/Chinese-LLaMA-Alpaca-2/blob/main/scripts/training/run_sft.sh
lora_r = 64
lora_alpha = 128
lora_dropout = 0.05
lora_target_modules = [
    "q_proj",
    "v_proj",
    "k_proj",
    "o_proj",
    "gate_proj",
    "down_proj",
    "up_proj",
]
# https://github.com/ymcui/Chinese-LLaMA-Alpaca-2/wiki/sft_scripts_zh#%E5%85%B3%E4%BA%8E%E6%98%BE%E5%AD%98%E5%8D%A0%E7%94%A8

"""
关于显存占用
默认配置训练llama，单卡24G会OOM，可以删去脚本中的--modules_to_save ${modules_to_save} \, 
即不训练embed_tokens和lm_head（这两部分参数量较大），只训练LoRA参数，单卡使用显存约21G。
"""
modules_to_save = [
    "embed_tokens",
    "lm_head"
]
# https://github.com/huggingface/peft/blob/main/src/peft/tuners/lora/config.py
config = LoraConfig(
    r=lora_r,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    target_modules=lora_target_modules,
    bias="none", # default is "none", this will change the results of original base model even when adapter is not active.
    task_type="CAUSAL_LM",
    inference_mode=False, ## default is false too # line 446
    # modules_to_save=modules_to_save ## default is "None". List of modules apart from LoRA layers to be set as trainable and saved in the final checkpoint.
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 159907840 || all params: 7089164288 || trainable%: 2.2556655975751596


In [24]:
!nvidia-smi

Wed Nov 22 15:13:10 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.06              Driver Version: 545.23.06    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A40                     Off | 00000000:AF:00.0 Off |                    0 |
|  0%   31C    P0              69W / 300W |  27578MiB / 46068MiB |      3%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Inference Before Training

In [23]:
prompt = f"""
: wareconn是什么?
:
""".strip()
print(prompt)

: wareconn是什么?
:


In [24]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
# generation_config.temperature = 0.0
generation_config.do_sample = False
# generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [25]:
generation_config

GenerationConfig {
  "bos_token_id": 1,
  "eos_token_id": 2,
  "max_new_tokens": 200,
  "pad_token_id": 2
}

In [25]:
%%time
device = "cuda:0"

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
    )
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

: wareconn是什么?
: wareconn是什么?
wareconn是什么?
: wareconn是什么?
: wareconn是什么?
: wareconn是什么?
: wareconn是什么?
: wareconn是什么?
: wareconn是什么?
: wareconn是什么?
: wareconn是什么?
: wareconn是什么?
: wareconn是什么?
: wareconn是什么?
: wareconn是什么?
: wareconn是什么?
: wareconn是什么?
: wareconn是什么?
: wareconn是什么?
: wareconn是什么?
: wareconn是什么?
: wareconn是什么?
: wareconn是什么?
: wareconn是什么?
: wareconn是什么?
: wareconn是什么?
: wareconn是什么?
: wareconn是什么?
: wareconn是什么?
: wareconn是什么?
CPU times: user 10.7 s, sys: 881 ms, total: 11.6 s
Wall time: 11.6 s


In [25]:
def generate_prompt(data_point):
    return f"""
: 在Wareconn系統上，{data_point["问题"]}
: {data_point["答案"]}
""".strip()


def generate_and_tokenize_prompt(data_point):
    full_prompt = generate_prompt(data_point)
    tokenized_full_prompt = tokenizer(full_prompt, padding=True, truncation=True, max_length=512)
    return tokenized_full_prompt

In [26]:
train_data = dataset.shuffle().map(generate_and_tokenize_prompt)

Map:   0%|          | 0/374 [00:00<?, ? examples/s]

In [27]:
train_data

Dataset({
    features: ['问题', '答案', 'input_ids', 'attention_mask'],
    num_rows: 374
})

## Training

In [28]:
OUTPUT_DIR = "experiments"

In [29]:
%load_ext tensorboard
%tensorboard --logdir experiments/runs

In [30]:
# https://github.com/huggingface/transformers/blob/main/src/transformers/training_args.py
# https://github.com/ymcui/Chinese-LLaMA-Alpaca-2/wiki/sft_scripts_zh
training_args = transformers.TrainingArguments(
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=1, # script is 8 but wiki is 1 (hf default is 1)
    num_train_epochs=2, #2 or 1
    learning_rate=1e-4,
    fp16=True,
    save_total_limit=3,
    save_strategy="steps",
    # eval_steps=250,
    # save_steps=500,
    logging_steps=10,
    logging_first_step=True,
    output_dir=OUTPUT_DIR,
    max_steps=100, # alpaca cn didn't set this # OOM error if it's too large
    optim="adamw_bnb_8bit", # 6BG of VRAM #"paged_adamw_8bit", # default is "adamw_torch" (requires 24 GB)
    # https://huggingface.co/docs/transformers/main/en/perf_train_gpu_one#optimizer-choice 
    lr_scheduler_type="cosine",
    warmup_ratio=0.03,
    weight_decay=0, # default is 0
    report_to="tensorboard",
    save_safetensors=False, # transformers' default is True
    # gradient_checkpointing= ## not specify but default is False
    ddp_find_unused_parameters=False, # default is False
    ddp_timeout=30000,
)

In [31]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=train_data,
    args=training_args,
    tokenizer=tokenizer,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
    # data_collator = DataCollatorForSupervisedDataset(tokenizer=tokenizer),
)
model.config.use_cache = False #line 421

In [33]:
%%time
trainer.train()

Step,Training Loss
1,3.830100
10,3.177300
20,2.899400
30,2.866200
40,3.053500
50,2.741500
60,2.259300
70,2.308400
80,2.542900
90,2.567300


CPU times: user 27.5 s, sys: 2.09 s, total: 29.6 s
Wall time: 29.6 s


TrainOutput(global_step=100, training_loss=2.7121367835998536, metrics={'train_runtime': 29.1727, 'train_samples_per_second': 3.428, 'train_steps_per_second': 3.428, 'total_flos': 399119270731776.0, 'train_loss': 2.7121367835998536, 'epoch': 0.27})

In [34]:
!nvidia-smi

Wed Nov 22 15:13:52 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.06              Driver Version: 545.23.06    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A40                     Off | 00000000:AF:00.0 Off |                    0 |
|  0%   41C    P0             168W / 300W |  44290MiB / 46068MiB |      7%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [35]:
model.save_pretrained("alpaca_cn_trained-model_warebot1.0")

## Load Trained Model

In [4]:
# PEFT_MODEL = f"ysw96/{peft_model_name}"
PEFT_MODEL = f"./alpaca_cn_trained-model_warebot1.0"

config = PeftConfig.from_pretrained(PEFT_MODEL)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    # quantization_config=bnb_config,
    device_map="auto",
    # trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token

model = PeftModel.from_pretrained(model, PEFT_MODEL)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [46]:
!nvidia-smi

Tue Nov 21 10:10:39 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.06              Driver Version: 545.23.06    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A40                     Off | 00000000:AF:00.0 Off |                    0 |
|  0%   31C    P0              69W / 300W |  45412MiB / 46068MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


## Inference

In [6]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
# generation_config.temperature = 0.0
generation_config.do_sample = False
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [7]:
DEVICE = "cuda:0"

In [38]:
encoding = tokenizer(prompt, return_tensors="pt").to(DEVICE)
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
    )
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

/usr/local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


: wareconn是什么?
: Wareconn是一个提供售后服务的云平台，包括保修管理、维修服务、数据分析、设备管理APP和派工管理APP等功能，旨在提高售后服务的效率和质量，帮助企业实现数字化流程掌控、多系统串接、智慧决策，持续提升企业营运绩效。
Wareconn平台提供安全可靠的售后服务云管理，支持多系统串接，实时掌控售后服务数据，推助工业 4.0 发展，赋能産品生命周期、价值链全周期管理与服务，持续提升企业营运绩效。
Wareconn平台服务全球客户，提供多语言支持，可在不同时区作业，确保与全球客户无时差沟通与服务。同时，Wareconn平台提供灵活的账号设定，可设定角色权限，确保数据安全性。
Wareconn平台致力于与客户共同成长，持续推助産品与服务的创新发展，为産品生命周期、价值链


In [8]:
def generate_response(question: str) -> str:
    prompt = f"""
: {question}
:
""".strip()
    encoding = tokenizer(prompt, return_tensors="pt").to(DEVICE)
    with torch.inference_mode():
        outputs = model.generate(
            input_ids=encoding.input_ids,
            attention_mask=encoding.attention_mask,
            generation_config=generation_config,
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    assistant_start = ":"
    response_start = response.find(assistant_start)
    return response[response_start + len(assistant_start) :].strip()

In [40]:
prompt = "wareconn是什么?"
print(generate_response(prompt))

wareconn是什么?
: Wareconn是一个提供售后服务的云平台，包括保修管理、维修服务、数据分析、设备管理APP和派工管理APP等功能，旨在提高售后服务的效率和质量，帮助企业实现数字化流程掌控、多系统串接、智慧决策，持续提升企业营运绩效。
Wareconn平台提供安全可靠的售后服务云管理，支持多系统串接，实时掌控售后服务数据，推助工业 4.0 发展，赋能産品生命周期、价值链全周期管理与服务，持续提升企业营运绩效。
Wareconn平台服务全球客户，提供多语言支持，可在不同时区作业，确保与全球客户无时差沟通与服务。同时，Wareconn平台提供灵活的账号设定，可设定角色权限，确保数据安全性。
Wareconn平台致力于与客户共同成长，持续推助産品与服务的创新发展，为産品生命周期、价值链


In [9]:
prompt = "派工APP能不能查看工程师工作数量统计？"
print(generate_response(prompt))

/usr/local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


派工APP能不能查看工程师工作数量统计？
: 主管可以在首页点击【行事历】查看所有工程师的接单数量,也可以点击【统计核算】查看所有工程师的月/年度工作数量统计。
工程师可以在首页点击【行事历】查看自己全部的接单数量,也可以点击【统计核算】查看自己月/年度的工作数量统计。
厂家可以在首页点击【行事历】查看所有服务中心的接单数量,也可以点击【统计核算】查看所有服务中心的月/年度工作数量统计。
客户可以在首页点击【行事历】查看所有服务中心的接单数量,也可以点击【统计核算】查看所有服务中心的月/年度工作数量统计。
厂家可以在首页点击【统计核算】查看所有服务中心的月/年度工作数量统计。
客户可以在首页点击【统计核算】查看所有服务中心的月/年度工作数量统计。
主管可以在首页点击【统计核算】查看所有服务中心的月/年度工作数量统计。
工程师


In [11]:
prompt = "wareconn有哪些功能?"
print(generate_response(prompt))

wareconn有哪些功能?
: Wareconn有以下主要功能：零件供应商管理、物料供应商管理、售后服务管理、工程支援服务管理、进度维护服务管理、货物收发管理、设备管理、人员指派服务管理、厂内维修服务管理、数据统计核算、紧急维保管理、资产履历管理、资产转移申请、资产保修管理、资产维保服务管理、资产替换管理、资产购置申请、资产信息设定、资产位置设定、资产进度设定、资产紧急维保、资产保修预测、资产保修费用核算、资产保修合约管理、资产保修费用核算、资产保修进度维护、资产保修费用预测、资产保修合约设定、资产保修费用核算、资产保修进度维护、资产保修费用预测、资产保修合约设定、资产保修费用核算、资产保修进度维护、资产保修费用预测、资产保修合约设定、资产保修费用核算、资产保修进度维护、资产保修费用预测、资产保修合约设定、资产保修费用核算


In [12]:
!nvidia-smi

Mon Nov 20 14:23:31 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.06              Driver Version: 545.23.06    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A40                     Off | 00000000:AF:00.0 Off |                    0 |
|  0%   37C    P0             197W / 300W |  28382MiB / 46068MiB |     99%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
